In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import math

# default='warn'
pd.options.mode.chained_assignment = None  
pd.set_option('display.max_columns', None)

%matplotlib inline

In [ ]:
# import data
df = pd.read_csv('data.csv',delimiter=',')
df.head()

In [ ]:
# check duplicated values
df.duplicated().all()

In [ ]:
# check null values 
df.isna().all()

In [ ]:
# check datatype
df.dtypes

In [ ]:
# country names
df.country.unique()

In [ ]:
# year
df.year.unique()

In [ ]:
# create sale data for each country and year 

def procreation(data,cntry,yr):
    aa = data.size_us.unique().tolist()
    aa.sort()
    dat = {'size_us':aa}
    df_new = pd.DataFrame(dat)
    d_n = pd.DataFrame(columns=range(1,13))
    df_new = df_new.append(d_n, ignore_index = True) 
    for mon in range(1,13):
        dd = data[(data['year']==yr) & (data['country']==cntry) & 
                       (data['month']==mon)]['size_us'].value_counts().tolist()
        dd1 = data[(data['year']==yr) & (data['country']==cntry) 
                   & (data['month']==mon)]['size_us'].value_counts().keys().tolist()
        dd11 = pd.DataFrame({'size':dd1,'count':dd})
        for ind , si in enumerate(df_new.size_us):
            for i,j in enumerate(dd11['size']):
                if si == j:
                    df_new[mon][ind] = dd11['count'][i]
    df_new.fillna(0,inplace=True)
    df_new.columns = ['size_us']+[str(x)+'_'+ str(yr) for x in ['jan','feb','mar','apr','may','jun',
                                                                'jul','aug','sep','oct','nov','dec']]
    return df_new


def procreation_shop(data,cntry,yr,shop):
    aa = data.size_us.unique().tolist()
    aa.sort()
    dat = {'size_us':aa}
    df_new = pd.DataFrame(dat)
    d_n = pd.DataFrame(columns=range(1,13))
    df_new = df_new.append(d_n, ignore_index = True) 
    for mon in range(1,13):
        dd = data[(data['year']==yr) & (data['country']==cntry) & 
                       (data['month']==mon) & (data['shop']==shop)]['size_us'].value_counts().tolist()
        dd1 = data[(data['year']==yr) & (data['country']==cntry) 
                   & (data['month']==mon) & (data['shop']==shop)]['size_us'].value_counts().keys().tolist()
        dd11 = pd.DataFrame({'size':dd1,'count':dd})
        for ind , si in enumerate(df_new.size_us):
            for i,j in enumerate(dd11['size']):
                if si == j:
                    df_new[mon][ind] = dd11['count'][i]
    df_new.fillna(0,inplace=True)
    df_new.columns = ['size_us']+[str(x)+'_'+ str(yr) for x in ['jan','feb','mar','apr','may','jun',
                                                                'jul','aug','sep','oct','nov','dec']]
    return df_new


def sum_all_country(data,cntry):
    count = 0
    for i in [2014,1015,2016]:
        if count == 0:
            df = procreation(data,cntry,i)
            df_OK = df.copy()
            count += 1
        else:
            df = procreation(data,cntry,i)
            df_OK = pd.merge(df_OK,df,on='size_us')
            
    return df_OK

def sum_all_shop(data,cntry,shop):
    count = 0
    for i in [2014,1015,2016]:
        if count == 0:
            df = procreation_shop(data,cntry,i,shop)
            df_OK = df.copy()
            count += 1
        else:
            df = procreation_shop(data,cntry,i,shop)
            df_OK = pd.merge(df_OK,df,on='size_us')
            
    return df_OK

#### We'll now generate the US sales data for the following years : 2014 , 2015 , 2016

In [ ]:
# US sale data
df_US = sum_all_country(df,'United States')

In [ ]:
df_US

#### We'll now generate the UK sales data for the following years : 2014 , 2015 , 2016

In [ ]:
# UK sale data
df_UK = sum_all_country(df,'United Kingdom')

In [ ]:
df_UK

#### We'll now generate the Canada sales data for the following years : 2014 , 2015 , 2016

In [ ]:
# Canada sale data
df_CANADA = sum_all_country(df,'Canada')

In [ ]:
df_CANADA

#### We'll now generate the Germany sales data for the following years : 2014 , 2015 , 2016

In [ ]:
# Germany sale data
df_GERMANY = sum_all_country(df,'Germany')

In [ ]:
df_GERMANY

### Get Sale Data for indiviual shops

### United States

In [ ]:
df[df['country'] == 'United States'].shop.unique().tolist()

In [ ]:
US1_hist = sum_all_shop(df,'United States','US1')

In [ ]:
US2_hist = sum_all_shop(df,'United States','US2')

In [ ]:
US3_hist = sum_all_shop(df,'United States','US3')

In [ ]:
US4_hist = sum_all_shop(df,'United States','US4')

In [ ]:
US5_hist = sum_all_shop(df,'United States','US5')

In [ ]:
US6_hist = sum_all_shop(df,'United States','US6')

In [ ]:
US7_hist = sum_all_shop(df,'United States','US7')

In [ ]:
US8_hist = sum_all_shop(df,'United States','US8')

In [ ]:
US9_hist = sum_all_shop(df,'United States','US9')

In [ ]:
US10_hist = sum_all_shop(df,'United States','US10')

In [ ]:
US11_hist = sum_all_shop(df,'United States','US11')

In [ ]:
US12_hist = sum_all_shop(df,'United States','US12')

In [ ]:
US13_hist = sum_all_shop(df,'United States','US13')

In [ ]:
US14_hist = sum_all_shop(df,'United States','US14')

In [ ]:
US15_hist = sum_all_shop(df,'United States','US15')

### United Kingdom

In [ ]:
df[df['country'] == 'United Kingdom'].shop.unique().tolist()

In [ ]:
UK1_hist = sum_all_shop(df,'United Kingdom','UK1')

In [ ]:
UK2_hist = sum_all_shop(df,'United Kingdom','UK2')

In [ ]:
UK3_hist = sum_all_shop(df,'United Kingdom','UK3')

In [ ]:
UK4_hist = sum_all_shop(df,'United Kingdom','UK4')

In [ ]:
UK5_hist = sum_all_shop(df,'United Kingdom','UK5')

### Canada

In [ ]:
df[df['country'] == 'Canada'].shop.unique().tolist()

In [ ]:
CAN1_hist = sum_all_shop(df,'Canada','CAN1')

In [ ]:
CAN2_hist = sum_all_shop(df,'Canada','CAN2')

In [ ]:
CAN3_hist = sum_all_shop(df,'Canada','CAN3')

In [ ]:
CAN4_hist = sum_all_shop(df,'Canada','CAN4')

In [ ]:
CAN5_hist = sum_all_shop(df,'Canada','CAN5')

In [ ]:
CAN6_hist = sum_all_shop(df,'Canada','CAN6')

In [ ]:
CAN7_hist = sum_all_shop(df,'Canada','CAN7')

### Germany

In [ ]:
df[df['country'] == 'Germany'].shop.unique().tolist()

In [ ]:
GER1_hist = sum_all_shop(df,'Germany','GER1')

In [ ]:
GER2_hist = sum_all_shop(df,'Germany','GER2')

In [ ]:
GER3_hist = sum_all_shop(df,'Germany','GER3')

### Statistical Analysis 

In [ ]:
# Function to calculate mean : overall , year : 2014,2015,2016 , standard_error , margin of error , 95% confidence interval
# and finally the total number of shoes should be stacked for each.

def calculate(data):
    '''
    
    We will perform T statistics here, because :
        Small sample data.
        We have sample data, unknown variance.
        We assume the distribution is normal.
        
        n = no of samples (36 - overall , 12 - each year)
        degree of freedom = n -1 = 35 - overall, 11 - each year
        95% confidence interval 
        alpha = 0.05
        alpha/2 = 0.025
        
        t(11),(0.025) = 2.201 [T-Score]
        t(35),(0.025) = 2.03 [T-Score-Overall]
        
    '''
    t_value = 2.201
    t_value_overall = 2.03
    data['mean_2014'] = data.iloc[:, np.r_[:,1:13]].mean(axis=1)
    data['mean_2015'] = data.iloc[:, np.r_[:,13:25]].mean(axis=1)
    data['mean_2016'] = data.iloc[:, np.r_[:,25:37]].mean(axis=1)
    data['mean_overall'] = data.iloc[:, np.r_[:,1:37]].mean(axis=1)
    data['std_error_2014'] = data.iloc[:, np.r_[:,1:13]].std(axis=1)/math.sqrt(12)
    data['std_error_2015'] = data.iloc[:, np.r_[:,13:25]].std(axis=1)/math.sqrt(12)
    data['std_error_2016'] = data.iloc[:, np.r_[:,25:37]].std(axis=1)/math.sqrt(12)
    data['std_error_overall'] = data.iloc[:, np.r_[:,1:37]].std(axis=1)/math.sqrt(36)
    data['margin_of_error_2014'] = data['std_error_2014'] * t_value
    data['margin_of_error_2015'] = data['std_error_2015'] * t_value
    data['margin_of_error_2016'] = data['std_error_2016'] * t_value
    data['margin_of_error_overall'] = data['std_error_overall'] * t_value_overall
    data['ci_low_2014'] = data['mean_2014'] - data['margin_of_error_2014']
    data['ci_high_2014'] = data['mean_2014'] + data['margin_of_error_2014']
    data['ci_low_2015'] = data['mean_2015'] - data['margin_of_error_2015']
    data['ci_high_2015'] = data['mean_2015'] + data['margin_of_error_2015']
    data['ci_low_2016'] = data['mean_2016'] - data['margin_of_error_2016']
    data['ci_high_2016'] = data['mean_2016'] + data['margin_of_error_2016']
    data['ci_low_overall'] = data['mean_overall'] - data['margin_of_error_overall']
    data['ci_high_overall'] = data['mean_overall'] + data['margin_of_error_overall']
    data['#_pairs_2014'] = np.ceil(data['ci_high_2014'])
    data['#_pairs_2015'] = np.ceil(data['ci_high_2015'])
    data['#_pairs_2016'] = np.ceil(data['ci_high_2016'])
    data['#_pairs_overall'] = np.ceil(data['ci_high_overall'])
    data['#_pairs_overall_minus2014'] = np.ceil(data['ci_high_2014'])
    data['#_pairs_overall_minus2015'] = np.ceil(data['ci_high_2015'])
    data['#_pairs_overall_minus2016'] = np.ceil(data['ci_high_2016'])
    
    return data

In [ ]:
# US
calculate(df_US)

In [ ]:
calculate(df_UK)

In [ ]:
calculate(df_CANADA)

In [ ]:
calculate(df_GERMANY)

### Similarly for each store we can calculate the confidence interval and mange our inventory efficiently

In [ ]:
calculate(US1_hist)

We can calculate for rest of the store similarly.

### Visualization for the results for better understanding.

In [ ]:
def visualize(data):
    figure(num=None, figsize=(10, 6), dpi=100, facecolor='w', edgecolor='k')
    plt.plot(data['size_us'],data['#_pairs_2014'],label="2014");
    plt.plot(data['size_us'],data['#_pairs_2015'],label="2015");
    plt.plot(data['size_us'],data['#_pairs_2016'],label="2016");
    plt.plot(data['size_us'],data['#_pairs_overall'],label="Overall");
    plt.legend(loc="upper left")
    plt.show()

In [ ]:
# United States
visualize(df_US)

In [ ]:
# United Kingdom
visualize(df_UK)

In [ ]:
# Canada
visualize(df_CANADA)

In [ ]:
# Germany
visualize(df_GERMANY)